# Laboratorium 5 - rekomendacje grafowe

## Przygotowanie

 * dataset i potrzebne biblioteki są dokładnie takie same jak na poprzednim laboratorium
 * pobierz i wypakuj dataset: https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
   * więcej możesz poczytać tutaj: https://grouplens.org/datasets/movielens/
 * [opcjonalnie] Utwórz wirtualne środowisko
 `python3 -m venv ./recsyslab5`
 * zainstaluj potrzebne biblioteki:
 `pip install numpy pandas sklearn gensim==3.8.3`

## Część 1. - przygotowanie danych

In [1]:
# importujemy wszystkie potrzebne pakiety

import math
import random
import numpy as np
import pandas

from gensim.models import Word2Vec

from sklearn.model_selection import train_test_split, KFold

In [56]:
SCORE_THRESHOLD = 4.0 # recenzje z co najmniej taka ocena wezmiemy pod uwage
VECTOR_SIZE = 20 # jak dlugie powinny byc wektory osadzen wierzcholkow
NEIGHBOURS_WINDOW = 11 # tylu sasiadow wezmiemy pod uwage w algorytmie Word2Vec (symetrycznie i wliczajac biezacy element)
PATH_LENGTH = 30 # dlugosc pojedynczej losowej sciezki
PATHS_COUNT_PER_NODE = 20 # liczba losowych sciezek zaczynajacych sie w kazdym z wierzcholkow

In [57]:
# wczytujemy oceny uytkownikow

ratings = pandas.read_csv('ratings.csv').drop(columns=['timestamp'])
ratings = ratings.where(ratings['rating'] >= SCORE_THRESHOLD).dropna()
# rozszerzamy ID tak, by sie nie powtarzaly
ratings['userId'] = ratings['userId'].apply(lambda x: 'u_' + str(int(x)))
ratings['movieId'] = ratings['movieId'].apply(lambda x: 'm_' + str(int(x)))
ratings

,userId,movieId,rating
0,u_1,m_1,4.0
1,u_1,m_3,4.0
2,u_1,m_6,4.0
3,u_1,m_47,5.0
4,u_1,m_50,5.0
...,...,...,...
100830,u_610,m_166528,4.0
100831,u_610,m_166534,4.0
100832,u_610,m_168248,5.0
100833,u_610,m_168250,5.0


In [58]:
# wczytujemy gatunki filmow

movies = pandas.read_csv('movies.csv').drop(columns=['title'])
movies['movieId'] = movies['movieId'].apply(lambda x: 'm_' + str(int(x)))
movies['genres'] = movies['genres'].apply(lambda x: x.split('|'))
movies_to_genres = movies.explode('genres')
movies_to_genres['genres'] = movies_to_genres['genres'].apply(lambda x: 'g_' + x.lower())
movies_to_genres = movies_to_genres.rename(columns = {'genres': 'genre'})
movies_to_genres

,movieId,genre
0,m_1,g_adventure
0,m_1,g_animation
0,m_1,g_children
0,m_1,g_comedy
0,m_1,g_fantasy
...,...,...
9738,m_193583,g_fantasy
9739,m_193585,g_drama
9740,m_193587,g_action
9740,m_193587,g_animation


In [59]:
users = ratings['userId'].unique()
movies = ratings['movieId'].unique()
genres = movies_to_genres['genre'].unique()

## Część 2. - spacer po grafie

In [61]:
# generujemy losowe sciezki w grafie
#   krawedzie reprezentowane sa w dwoch macierzach - ratings i movies
#   w wersji podstawowej wszystkie krawedzie traktujemy jako niewazone i nieskierowane
#   mozliwe ulepszenia:
#    - rozwazenie krawedzi skierowanych
#    - uwzglednienie wag krawedzi (ocen uzytkownikow)
#    - jakas forma normalizacji - obnizenia wag wierzcholkow o wysokich stopniach
#    - Node2Vec - parametry P i Q
# wynikiem powinna byc lista list - kazda z tych list zawiera kolejne ID wierzcholkow na sciezce
from enum import Enum
from random import random, choice

class Prev(Enum):
  User = 1
  Movie = 2
  Genre = 3

def get_paths(node_name, prev, ratings, movies_to_genres, paths_per_node, path_length):
    paths = []
    for path_number in range(paths_per_node):
      path = [node_name]  
      for path_step in range(path_length):
        # print(f"nodename_main: {node_name}")
        if prev == Prev.User:
          print(f"nodename_user: {node_name}")
          # print(ratings[:,0])
          possible_nodes = np.where(ratings[:,0] == node_name)
          # print(ratings[possible_nodes])
          next_node = choice(ratings[possible_nodes])[1]
          prev = Prev.Movie
        elif prev == Prev.Movie:
          print(f"nodename_movie: {node_name}")
          if random() < .5: #go to user
            possible_nodes = np.where(ratings[:,1] == node_name)
            # print(ratings[possible_nodes])
            # print(choice(ratings[possible_nodes])[0])
            next_node = choice(ratings[possible_nodes])[0]
            prev = Prev.User
          else: #go to genre
            possible_nodes = np.where(movies_to_genres[:,0] == node_name)
            next_node = choice(movies_to_genres[possible_nodes])[1]
            prev = Prev.Genre
        elif prev == Prev.Genre:
          print(f"nodename_genre: {node_name}")
          possible_nodes = np.where(movies_to_genres[:,1] == node_name)
          next_node = choice(movies_to_genres[possible_nodes])[0]
          prev = Prev.Movie        
        node_name = next_node
        path.append(next_node)
      paths.append(path)
    return paths



def generate_walks(ratings, movies_to_genres, paths_per_node, path_length):
    paths = []
    # print(f"ratings: \n{ratings}")
    # print(f"movies to genres: \n{movies_to_genres}")
    # print(f"pathspernode: \n{paths_per_node}")
    # print(f"path_length: \n{path_length}")
    # print(f"unique users: {ratings['userId'].unique()}")
    # print(f"unique movies: {ratings['movieId'].unique()}")
    # print(f"unique genres: {movies_to_genres['genre'].unique()}")
    unique_users = ratings['userId'].unique()
    unique_movies = ratings['movieId'].unique()
    unique_genres = movies_to_genres['genre'].unique()
    ratings = ratings.to_numpy()
    movies_to_genres = movies_to_genres.to_numpy()
    
    possible_nodes = np.where(ratings[:,1] == 'm_108090')
    print(ratings[possible_nodes])
    for i in ratings:
      if i[1] == 'm_108090':
        print(i)
    for user_id in unique_users:
      path = get_paths(user_id, Prev.User, ratings, movies_to_genres, paths_per_node, path_length)
      paths.extend(path)
    for movie_id in unique_movies:
      path = get_paths(movie_id, Prev.Movie, ratings, movies_to_genres, paths_per_node, path_length)
      paths.extend(path)
    for genre in unique_genres:
      path = get_paths(genre, Prev.Genre, ratings, movies_to_genres, paths_per_node, path_length)
      paths.extend(path)

    # ...
    return paths
    
walks = generate_walks(ratings, movies_to_genres, PATHS_COUNT_PER_NODE, PATH_LENGTH)

[]
nodename_user: u_1
nodename_movie: m_2991
nodename_genre: g_action
nodename_movie: m_187593
nodename_user: u_331
nodename_movie: m_30749
nodename_user: u_460
nodename_movie: m_1704
nodename_genre: g_romance
nodename_movie: m_2774
nodename_genre: g_comedy
nodename_movie: m_27826
nodename_genre: g_comedy
nodename_movie: m_7366
nodename_genre: g_drama
nodename_movie: m_6170
nodename_genre: g_adventure
nodename_movie: m_4367
nodename_genre: g_action
nodename_movie: m_76175
nodename_genre: g_adventure
nodename_movie: m_62376
nodename_genre: g_adventure
nodename_movie: m_2275
nodename_genre: g_action
nodename_movie: m_83349
nodename_user: u_534
nodename_movie: m_89745
nodename_user: u_62
nodename_movie: m_2959
nodename_user: u_103
nodename_movie: m_1230
nodename_genre: g_comedy
nodename_movie: m_110541
nodename_genre: g_comedy
nodename_movie: m_7033
nodename_genre: g_comedy
nodename_movie: m_142997
nodename_genre: g_animation
nodename_movie: m_62999
nodename_user: u_210
nodename_movie: m_

IndexError: ignored

## Część 3. - obliczenie osadzeń

In [ ]:
# trenujemy model
#   zauwaz, ze wszystkie trzy rodzaje wierzcholkow beda reprezentowane tak samo, w tej samej przestrzeni

model = Word2Vec(sentences=walks, size=VECTOR_SIZE, window=NEIGHBOURS_WINDOW, min_count=1, workers=4)
embeddings = model.wv

## Część 4. - rekomendacje i zastosowania

In [ ]:
PULP_FICTION = 'm_296'
TOY_STORY = 'm_1'
PLANET_OF_THE_APES = 'm_2529'

In [ ]:
# wyszukajmy K najpodobniejszych filmów do danego
# porownaj wyniki dla odleglosci euklidesowej i cosinuswej, np. na trzech powyzszych filmach

def euclidian_distance(i, j):
    pass

def cosine_distance(i, j):
    pass

def k_most_similar_movies(movie_id, K, embeddings, distance_fun):
    # ...
    return k_most_similar

k_most_similar_movies(PULP_FICTION, 5, embeddings, cosine_distance)

In [ ]:
# wyszukajmy k filmow najblizszych uzytkownikowi
# wykorzystaj funkcje z poprzedniej komorki

def k_best_movies_for_user(user_id, K, embeddings, distance_fun):
    # ...
    return k_best_movies

In [ ]:
# sprobujmy czegos bardziej skomplikowanego
#   znajdz ulubiony gatunek filmowy uzytkownika
#   a nastepnie zaproponuj K filmow z tego gatunku - ale nie tych najblizszych uzytkownikowi
#   (zaproponuj, w jaki sposob dobrac filmy interesujace, ale nie z najblizszego otoczenia)

def k_from_favourite_genre(user_id, K, embeddings, distance_fun):
    # ...
    return k_from_genre

In [ ]:
# Na koniec najbardziej skomplikowany algorytm - odpowiednik "Radia utworu" w Spotify.
#   Zaczynamy od jednego filmu, a nastepnie wyznaczamy kolejne, wedrujac po przestrzeni, w ktorej wszystkie elementy sa osadzone.
#   Zaproponuj, jak zdefiniowac podzbior filmow, z ktorych bedziemy wybierac (np. filmy odlegle o min. a i max. b od danego)
#   oraz jak generowac kolejny skok (tak, zeby seria rekomendacji nie byla zbyt monotonna, ale rownoczesnie zgodna z gustem uzytkownika)

def get_playlist(start_movie_id, user_id, K, embeddings):
    # ...
    return playlist